In [2]:
using JuMP
using Gurobi
using CSV
using DataFrames

In [3]:
const GRB_ENV = Gurobi.Env(output_flag=1);

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-06


In [4]:
centers = CSV.File("HW3 data/centers.csv",header=0) |> Tables.matrix;
stations = CSV.File("HW3 data/stations.csv",header=0) |> Tables.matrix;
landfills = CSV.File("HW3 data/landfills.csv",header=0) |> Tables.matrix;
q = CSV.File("HW3 data/q.csv",header=0) |> Tables.matrix;

centers2 = CSV.File("HW3 data/centers2.csv",header=0) |> Tables.matrix;
stations2 = CSV.File("HW3 data/stations2.csv",header=0) |> Tables.matrix;
landfills2 = CSV.File("HW3 data/landfills2.csv",header=0) |> Tables.matrix;
q2 = CSV.File("HW3 data/q2.csv",header=0) |> Tables.matrix;

centers_all = [centers;centers2];
stations_all = [stations;stations2];
landfills_all = [landfills;landfills2];
q_all = [q;q2];

n1 = size(centers)[1]
s1 = size(stations)[1]
m1 = size(landfills)[1]
n2 = size(centers2)[1]
s2 = size(stations2)[1]
m2 = size(landfills2)[1]
n_all = n1+n2;
s_all = s1+s2;
m_all = m1+m2;

LoadError: ArgumentError: "HW3 data/centers.csv" is not a valid file or doesn't exist

In [ ]:
model = Model(Gurobi.Optimizer)

k = 5 # number of landfills to build
# Calculate distance matrix
distances = [sqrt((centers[i, 1] - landfills[j, 1])^2 + (centers[i, 2] - landfills[j, 2])^2) for i in 1:n1, j in 1:m1]
distances_center_to_stations = [sqrt((centers[i, 1] - stations[k, 1])^2 + (centers[i, 2] - stations[k, 2])^2)  for i in 1:n1, k in 1:s1]
distances_stations_to_landfils = [sqrt((stations[k, 1]- landfills[j, 1])^2 + (stations[k, 2]- landfills[j, 2])^2) for k in 1:s1, j in 1:m1]

# Additional Costs
station_cost = 10000  # daily operating cost per transfer station
compact_cost = 0.50  # per mile-ton cost for compacted waste
uncomp_cost = 1.0  # per mile-ton cost for uncompacted waste

# Decision Variables from Part A
@variable(model, x[1:m1], Bin)   # 1 if landfill j is selected, 0 otherwise
@variable(model, y[1:n1, 1:m1] >= 0)  # amount of waste sent from center i to landfill j

# Additional Decision Variables
# @variable(model, station[1:s1], Bin) # 1 if compact station is builts at location k
@variable(model, z[1:s1], Bin)  # 1 if transfer station is built at location j
@variable(model, w[1:n1, 1:s1] >= 0)  # waste sent from center i to transfer station k as uncompacted waste
@variable(model, t[1:s1, 1:m1] >= 0)  # waste sent from transfer station k to landfill i as compacted waste

# Update Objective: Include construction and transportation costs
@objective(model, Min, sum(distances[i, j] * y[i, j] * uncomp_cost for i in 1:n1, j in 1:m1) +      # uncompressed distance cost
                        sum(distances_center_to_stations[i, k] * w[i, k] * uncomp_cost for i in 1:n1, k in 1:s1) +    # compressed distance cost, on way to station
                        sum(distances_stations_to_landfils[k, j] * t[k, j] * compact_cost for k in 1:s1, j in 1:m1) +    # compressed distance cost, on way to landfill
                        sum(station_cost * z[k] for k in 1:s1))                                     # station cost

# Original Constraints - Part A
@constraint(model, sum(x) == k)  # Select exactly k landfills
# @constraint(model, [i=1:n1], sum(y[i, j] for j in 1:m1) == q[i])  # Meet waste requirements at each center
# @constraint(model, [j=1:m1], sum(y[i, j] for i in 1:n1) <= 100000 * x[j])  # Capacity constraint

# Additional Constraints
@constraint(model, [i=1:n1], sum(y[i, j] * x[j] for j in 1:m1) + sum(w[i, k] * z[k] for k in 1:s1) == q[i])  # Waste from each center
@constraint(model, [k=1:s1], sum(w[i, k] for i in 1:n1) <= 2000 * z[k])  # Transfer station capacity
# @constraint(model, [j=1:s1, k=1:m1], t[j, k] <= 2000 * z[j])  # Compacted waste limits
@constraint(model, [k=1:s1], sum(w[i, k] * z[k] for i in 1:n1) == sum(t[k, i] * z[k] for i in 1:m1))

# Solve
optimize!(model)

# Results
selected_landfills = [j for j in 1:m1 if value(x[j]) > 0.5]
total_cost = objective_value(model)
selected_transfer_stations = [j for j in 1:s1 if value(z[j]) > 0.5]

# calculate distances of trash that either went to center or didnt
distance_c_s = sum(value(distances_center_to_stations[i, j]) for i in 1:n1, j in 1:s1 if value(z[j]) > 0.5)
distances_s_l = sum(value(distances_stations_to_landfils[i,j]) for i in 1:s1, j in 1:m1 if value(z[i])>0.5)
ditances_none =  sum(value(distances[i, j]) for i in 1:n1, j in 1:m1)
distance_traveled_by_waste = distance_c_s + distances_s_l + ditances_none

println("Selected Landfills: ", selected_landfills)
println("Total Cost: ", total_cost)
println("Selected Transfer Stations: ", selected_transfer_stations)
println("Distance Traveled by Waste: ", distance_traveled_by_waste)